In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd 
import random
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.svm import SVC 
from sklearn.preprocessing import StandardScaler 
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA

In [2]:
def my_preprocess(train):
    train['Geography'] = train['Geography'].map({'S0':0, 'S1':1, 'S2':2})
    train['Gender'] = train['Gender'].map({'Male':0, 'Female':1})
    
    Balance = train['Balance'].tolist()
    Balance_range = []
    for i in range(len(Balance)):
        if Balance[i]>100000 and Balance[i]<140000:
            Balance_range.append(1)
        elif Balance[i]>50000 and Balance[i]<75000:
            Balance_range.append(2)
        else:
            Balance_range.append(0)
    train['Balance_range'] = Balance_range

    EstimatedSalary = train['EstimatedSalary'].tolist()
    EstimatedSalary_range = []
    for i in range(len(EstimatedSalary)):
        if EstimatedSalary[i]>=60000 and EstimatedSalary[i]<80000:
            EstimatedSalary_range.append(1)
        elif EstimatedSalary[i]>=80000 and EstimatedSalary[i]<100000:
            EstimatedSalary_range.append(2)
        elif EstimatedSalary[i]>=100000 and EstimatedSalary[i]<120000:
            EstimatedSalary_range.append(3)
        elif EstimatedSalary[i]>=12000 and EstimatedSalary[i]<140000:
            EstimatedSalary_range.append(4)
        else:
            EstimatedSalary_range.append(0)
    train['EstimatedSalary_range'] = EstimatedSalary_range

    Age = train['Age'].tolist()
    Age_range = []
    for i in range(len(Age)):
        if Age[i]>=15 and Age[i]<22:
            Age_range.append(1)
        elif Age[i]>=22 and Age[i]<35:
            Age_range.append(2)
        elif Age[i]>=35 and Age[i]<50:
            Age_range.append(3)
        elif Age[i]>=50 and Age[i]<65:
            Age_range.append(4)
        else:
            Age_range.append(0)
    train['Age_range'] = Age_range
    
    return train

In [3]:
train = pd.read_csv('train.csv')

#data preprocessing
target = 'Exited'
scaler = StandardScaler()
train = my_preprocess(train)
predictors = [x for x in train.columns if x not in \
              ['U', 'RowNumber', 'CustomerId', 'Surname', 'Exited',\
               ]]
train[predictors] = pd.DataFrame(train[predictors],dtype=np.float)
scaler.fit(train[predictors])
train[predictors] = scaler.transform(train[predictors])

#using PCA to reduce dimensions
pca = PCA(n_components=6)
pca.fit(train[predictors])
pca_train = pca.transform(train[predictors])

pca_train = pd.DataFrame(pca_train)
pca_train.head()

,0,1,2,3,4,5
0,0.388243,0.516653,-0.194090,2.227349,-0.849490,1.633929
1,-1.066476,0.671889,2.081689,-1.011258,0.884035,-0.189904
2,0.735596,1.842888,-2.107550,0.716662,0.154396,-0.558477
3,0.777562,0.628221,-0.343723,-1.284046,0.607001,1.094961
4,1.242081,1.541231,0.146290,2.462832,-0.986208,1.404786


In [5]:
X_train, X_test, y_train, y_test = train_test_split(pca_train, train['Exited'], test_size = 0.20) 

svclassifier = SVC(kernel='rbf', gamma=1.0, C=1.0) 
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)  

print(classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

           0       0.83      0.97      0.89      1264
           1       0.71      0.23      0.35       336

   micro avg       0.82      0.82      0.82      1600
   macro avg       0.77      0.60      0.62      1600
weighted avg       0.80      0.82      0.78      1600

